# Looking Inside Large Language Models

## Initial Setup

In [1]:
# Initial imports
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2025-12-01 13:54:43.021450: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 13:54:43.025139: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 13:54:43.036408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764597283.055508     409 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764597283.061186     409 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764597283.075502     409 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Loading the LLM

In [2]:
device: str = "cpu"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="microsoft/Phi-3-mini-4k-instruct"
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="microsoft/Phi-3-mini-4k-instruct",
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=False,
)

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=250,
    do_sample=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


## An Overview of Transformer Models

### The Inputs and Outputs of a Trained Transformer LLM

In [3]:
prompt: str = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."
output = generator(prompt, max_new_tokens=50)

print(output[0]["generated_text"])

 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my sincerest apologies for the unfortunate incident that occurred


In [4]:
%%time

prompt: str = "Please, can you give me a brief overview of life and works of Gary Marcus in the Field of AI? Give me a list of bullet points of his main ideas."
output = generator(prompt, max_new_tokens=500)

print(output[0]["generated_text"])



**Solution 1:**

- Gary Marcus is a cognitive scientist and psychologist known for his work in the field of artificial intelligence (AI).
- He has been a professor at New York University and is currently a researcher at Microsoft.
- Marcus has written several books on AI, including "Guitar Zero," "The Baby in the Box," and "Kluge: The Haphazard Construction of the Human Mind."
- He has been a vocal critic of deep learning and has advocated for a more symbolic approach to AI.
- Marcus has argued that deep learning models are often "black boxes" that lack interpretability and transparency.
- He has proposed that AI should be built on a foundation of symbolic reasoning and logic, rather than solely relying on statistical methods.
- Marcus has also been involved in research on language acquisition and cognitive development in children.
- He has contributed to the development of the "baby brain" theory, which suggests that the human brain is a "kluge" or a patchwork of evolutionary soluti

### The Components of the Forward Pass

In [5]:
# Print model architecture
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
 

### Choosing a Single Token from the Probability Distribution (Sampling / Decoding)